<a href="https://colab.research.google.com/github/ykitaguchi77/Whisper_transcription/blob/main/Whisper_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Whisper Fine Tuning**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
sound_path = "/content/drive/MyDrive/Deep_learning/Transcription/2023-07-16_15-47-33.WAV"

In [3]:
!pip install pydub

In [4]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os
import shutil

sound_path = "/content/drive/MyDrive/Deep_learning/Transcription/2023-07-16_15-47-33.WAV"
source = AudioSegment.from_file(sound_path, format="wav")

chunks = split_on_silence(source,
                          min_silence_len = 100,
                          silence_thresh = -40,
                          keep_silence = 200)

outputDirname="/content/drive/MyDrive/Deep_learning/Transcription/data"

if os.path.exists(outputDirname):
  shutil.rmtree(outputDirname)

os.makedirs(outputDirname,exist_ok=True)

fileList=[]
current_chunk = None

for i, chunk in enumerate(chunks):
    if current_chunk is None:
      current_chunk = chunk
    else:
      current_chunk+=chunk

    if len(current_chunk)/1000>30:
      num_of_chunk = int(len(current_chunk)/30000)
      for j in range(num_of_chunk+1):
        outFilePath = f'{outputDirname}/out_{i + 1}_{j + 1}.wav'
        current_chunk[j*30000:(j+1)*30000].export(outFilePath, format="wav")
        fileList.append(outFilePath)
      current_chunk = None

In [5]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-iwgat7s8
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-iwgat7s8
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.2 MB/s eta 0:0

In [6]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00


In [8]:
from datasets import Dataset,Audio
import whisper
from whisper.audio import N_FRAMES, pad_or_trim, log_mel_spectrogram
from whisper.tokenizer import get_tokenizer
import torch

model = whisper.load_model("large")
datasets = Dataset.from_dict({"audio": fileList}).cast_column("audio", Audio())

# predict_data = []
# for i, f in enumerate(fileList):
#   mel = log_mel_spectrogram(f)
#   segment = pad_or_trim(mel, N_FRAMES).to(model.device).to(torch.float16)
#   result = model.decode(segment)
#   tokenizer = get_tokenizer(multilingual=True, language="ja", task="transcribe")
#   outputText = tokenizer.decode(result.tokens)
#   predict_data.append({**datasets[i], "sentence":outputText})

predict_data = []
for i, f in enumerate(fileList):
  mel = log_mel_spectrogram(f, n_mels=128)  # n_mels を 128 に変更
  segment = pad_or_trim(mel, N_FRAMES).to(model.device).to(torch.float16)
  result = model.decode(segment)
  tokenizer = get_tokenizer(multilingual=True, language="ja", task="transcribe")
  outputText = tokenizer.decode(result.tokens)
  predict_data.append({**datasets[i], "sentence":outputText})

OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 17.06 MiB is free. Process 17968 has 14.73 GiB memory in use. Of the allocated memory 14.08 GiB is allocated by PyTorch, and 552.07 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import gspread
from google.auth import default
from google.colab import auth
from subprocess import getoutput
!apt-get install xattr > /dev/null

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

spread_sheet_dir_id = getoutput("xattr -p 'user.drive.id' " + "'" + outputDirname[:-5] + "'")
ss = gc.create('Transcription_Dataset', folder_id=spread_sheet_dir_id)

ss.append_row(['url', 'colab_path','sampling_rate','correct','whisper'])

rows = []
for data in predict_data:
  file_id = getoutput("xattr -p 'user.drive.id' " + "'" +  data["audio"]["path"] + "'")
  url = f"https://drive.google.com/file/d/{file_id}"
  colab_path = data["audio"]["path"]
  sampling_rate = data["audio"]["sampling_rate"]
  whisper_text = data["sentence"]
  rows.append([url, colab_path, sampling_rate, "", whisper_text])

ss.values_append("シート1",{"valueInputOption":"USER_ENTERED"},{"values":rows})

In [10]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

import torch
torch.cuda.empty_cache()

In [11]:
!pip install pydub

import os
import shutil
import pandas as pd
from pydub import AudioSegment
from pydub.silence import split_on_silence
import whisper

# 音声ファイルのパスを指定
sound_path = "/content/drive/MyDrive/Deep_learning/Transcription/2023-07-16_15-47-33.WAV"

# 音声ファイルを読み込む
audio = AudioSegment.from_wav(sound_path)

# 無音部分で音声を分割
chunks = split_on_silence(audio,
                          min_silence_len = 100, # 無音部分の長さ(ms)
                          silence_thresh = -40, # 無音判定の閾値(dB)
                          keep_silence = 200) # 無音時間をどのくらい残すか(ms)

# 分割後の音声ファイルを保存するディレクトリを作成
output_dir = "/content/drive/MyDrive/Deep_learning/Transcription/data"
if os.path.exists(output_dir):
  shutil.rmtree(output_dir)

os.makedirs(output_dir, exist_ok=True)

# Whisperモデルをロード
model = whisper.load_model("large")

# 結果を保存するデータフレームを作成
results = []

# 30秒ごとにchunkを連結し、Whisperで文字起こしを行う
current_chunk = None
file_list = []

for i, chunk in enumerate(chunks):
    if current_chunk is None:
      current_chunk = chunk
    else:
      current_chunk += chunk

    # current_chunkを連結したものが30秒以上になるなら出力
    if len(current_chunk) / 1000 > 30:
      # 30秒区切りにする場合何個になるかを計算
      num_of_chunk = int(len(current_chunk) / 30000)
      # 30秒ごとに区切って出力
      for j in range(num_of_chunk + 1):
        segment_path = f'{output_dir}/segment_{i + 1}_{j + 1}.wav'
        current_chunk[j * 30000:(j + 1) * 30000].export(segment_path, format="wav")
        file_list.append(segment_path)

        # Whisperで文字起こしを行う
        result = model.transcribe(segment_path)

        # 結果をデータフレームに追加
        results.append({"file_path": segment_path, "transcript": result["text"]})

      current_chunk = None

# 結果をcsvファイルに保存
df = pd.DataFrame(results)
df.to_csv("/content/drive/MyDrive/Deep_learning/Transcription/transcription_results.csv", index=False)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 17.06 MiB is free. Process 17968 has 14.73 GiB memory in use. Of the allocated memory 14.08 GiB is allocated by PyTorch, and 545.10 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)